In [140]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

XPATH = By.XPATH

# url = 'https://www.tinkoff.ru/invest/bonds/?start=0&end=12&country=Russian&orderType=Desc&sortType=ByYieldToClient'
url = 'https://www.tinkoff.ru/invest/bonds/?start=0&end=2400&country=Russian&orderType=Desc&sortType=ByYieldToClient'

In [153]:
class Obligations_parser:
    def __init__(self, driver):
        self.driver = driver
        self.driver.maximize_window()
        self.driver.get(url)
        # self.obligations_data = pd.DataFrame(columns = ['Название', "Код", 'Ссылка', "Срок(месяца)", "Цена", "Номинал", "Выплат в год", "Величина купона(-13%)", "Прибыль в год(-13% с купона)", "Доходность(все время)", "Доходность(год)", "Амортизация", 'Рейтинг'])
        self.obligations_data = pd.DataFrame()
    

    def next_layer(self):
        next_page_class = 'Pagination-module__item_ALrvz Pagination-module__item_size_s_voSgR Pagination-module__item_arrow_mCpmW'
        # retry_counter = 0
        
        try:
            self.driver.find_element(XPATH, f'//a[@class = "{next_page_class}"]').click()
            self.elements_iterator()
            return True

        except Exception:
            print('Все страницы пройдены.')

            # self.elements_iterator()
            return(self.obligations_data)
        


    
    def grab_paper_info(self, paper_url : str = ''):
        if paper_url:
            self.driver.execute_script("window.open('');")
            self.driver.switch_to.window(self.driver.window_handles[-1])
            self.driver.get(paper_url)
        
        else:
            self.driver.refresh()

        paper_data = {}

        try:
            paper_data.update({'Название' : [self.driver.find_element(XPATH, '//span[@class = "SecurityHeader__showName_iw6qC"]').text]})
            
            paper_data.update({'Код' : [self.driver.find_element(XPATH, '//span[@class = "SecurityHeader__ticker_j7fZW"]').text]})


            paper_time = self.driver.find_element(XPATH, '//div[@class = "SecurityHeader__panelText_KDJdO"]').text
            paper_time = paper_time[paper_time.index('з') + 3 : ]
            paper_data.update({'Срок(месяца)' : [int(paper_time[ : paper_time.index(' ')])]})
            del paper_time

            paper_data.update({'Цена' : [float(self.driver.find_element(XPATH, '//div[@class = "SecurityInvitingScreen__price_FSP8P"]').text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -1])]})
            
            paper_data.update({'Рейтинг' : [self.driver.find_element(XPATH, '//div[@class = "SecurityHeader__panel_itBzT SecurityHeader__desktop_dL7RD"]/div[@class = "SecurityHeader__panelText_KDJdO"]').text]})

            paper_datas = self.driver.find_elements(XPATH, '//td[@class = "Table-module__cell_RJ0qz"]')



            for counter in range(1, len(paper_datas) + 1, 2):
                match paper_datas[counter - 1].text:
                    case "Величина купона":
                        paper_coupon = float(paper_datas[counter].text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -2])
                        paper_coupon -= paper_coupon * 0.13     # Вычитаем налог 13% на купоны

                        paper_data.update({'Величина купона(-13%)' : [paper_coupon]})
                    
                    case "Номинал":
                        paper_data.update({'Номинал' : [float(paper_datas[counter].text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -1])]})
                    
                    case "Количество выплат в год":
                        paper_data.update({'Выплат в год' : [int(paper_datas[counter].text)]})
                        paper_data.update({'Доходность(год)' : [paper_data['Выплат в год'][0] * paper_data['Величина купона(-13%)'][0]]})
                        # paper_data.update({'Доходность(все время)' : [paper_data['Срок(месяца)'][0] // 2 * paper_data['Доходность(год)'][0]]})
                        paper_data.update({'Доходность(все время)' : [paper_data['Срок(месяца)'][0] // paper_data['Выплат в год'][0] * paper_data['Величина купона(-13%)'][0]]})

                    
                    case 'Амортизация':
                        paper_data.update({'Амортизация' : [True] if paper_datas[counter] == 'Да' else [False]})
                    

            print(paper_data)

            paper_data.update({'Ссылка' : [paper_url]})

            # print(pd.DataFrame(paper_data))
            # print(pd.DataFrame.from_dict(paper_data), )

            

            self.obligations_data = pd.concat([self.obligations_data, pd.DataFrame(paper_data)])

            
            self.driver.close()

        except Exception:
            self.driver.close()
            time.sleep(10)
            self.grab_paper_info()

    def elements_iterator(self):

        link_buttons_class = 'Link-module__link_yQVl1 Link-module__link_theme_default_mkRhf'

        # Использую тр и тд, поскольку на сайте в строках тр 5 ссылок с нужным адресом, потому просто по классу ссылки не выйдет найти одно совпадение - придется потом чистить список. Приходится потихоьнку находить элементы.
        tr_link_buttons_class = 'Table-module__row_Qlwsh Table-module__row_clickable_FeO1O'
        # td_link_buttons_class = 'Table-module__cell_RJ0qz'

        papers_rows = self.driver.find_elements(XPATH, f'//tr[@class = "{tr_link_buttons_class}"]')
        
        for paper in papers_rows:
            self.driver.switch_to.window(self.driver.window_handles[-1])

            paper_link = paper.find_element(XPATH, 'td').find_element(XPATH, 'a').get_attribute('href')

            self.grab_paper_info(paper_link)
            

    
        return(self.obligations_data)

            

        # self.next_layer()



        # for paper in papers_rows:
        #     paper = paper.find_element(XPATH, '//td').find_element(XPATH, f'//a[@class = "{link_buttons_class}"]')
        #     print(paper.get_attribute('href'))




In [151]:
a = webdriver.Chrome()

In [154]:
parser_object = Obligations_parser(a)

In [155]:
parser_object.elements_iterator()

{'Название': ['Ferroni выпуск 1'], 'Код': ['RU000A103XP8'], 'Срок(месяца)': [24], 'Цена': [749.9], 'Рейтинг': ['Низкий'], 'Величина купона(-13%)': [9.222], 'Номинал': [1000.0], 'Выплат в год': [12], 'Доходность(год)': [110.66399999999999], 'Доходность(все время)': [18.444]}
{'Название': ['Феррони БО-01'], 'Код': ['RU000A1053R3'], 'Срок(месяца)': [22], 'Цена': [934.0], 'Рейтинг': ['Низкий'], 'Величина купона(-13%)': [11.745], 'Номинал': [1000.0], 'Выплат в год': [12], 'Доходность(год)': [140.94], 'Доходность(все время)': [11.745]}
{'Название': ['СПМК БО-02'], 'Код': ['RU000A106FN3'], 'Срок(месяца)': [5], 'Цена': [966.8], 'Рейтинг': ['Низкий'], 'Величина купона(-13%)': [30.884999999999998], 'Номинал': [1000.0], 'Выплат в год': [4], 'Доходность(год)': [123.53999999999999], 'Доходность(все время)': [30.884999999999998]}
{'Название': ['Агрофирма-племзавод Победа выпуск 1'], 'Код': ['RU000A102S23'], 'Срок(месяца)': [4], 'Цена': [939.1], 'Рейтинг': ['Низкий'], 'Величина купона(-13%)': [21.141

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=117.0.5938.92)
Stacktrace:
	GetHandleVerifier [0x00007FF67E537892+54818]
	(No symbol) [0x00007FF67E4A6AC2]
	(No symbol) [0x00007FF67E35DA3B]
	(No symbol) [0x00007FF67E33DEE8]
	(No symbol) [0x00007FF67E3C3237]
	(No symbol) [0x00007FF67E3D6C6F]
	(No symbol) [0x00007FF67E3BE883]
	(No symbol) [0x00007FF67E393691]
	(No symbol) [0x00007FF67E3948D4]
	GetHandleVerifier [0x00007FF67E89B992+3610402]
	GetHandleVerifier [0x00007FF67E8F1860+3962352]
	GetHandleVerifier [0x00007FF67E8E9D4F+3930847]
	GetHandleVerifier [0x00007FF67E5D3646+693206]
	(No symbol) [0x00007FF67E4B1628]
	(No symbol) [0x00007FF67E4AD934]
	(No symbol) [0x00007FF67E4ADA62]
	(No symbol) [0x00007FF67E49E113]
	BaseThreadInitThunk [0x00007FF9F1477344+20]
	RtlUserThreadStart [0x00007FF9F1D426B1+33]


In [156]:
df = parser_object.obligations_data

In [157]:
df

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Ссылка
0,Ferroni выпуск 1,RU000A103XP8,24,749.9,Низкий,9.222,1000.0,12,110.664,18.444,https://www.tinkoff.ru/invest/bonds/RU000A103XP8/
0,Феррони БО-01,RU000A1053R3,22,934.0,Низкий,11.745,1000.0,12,140.940,11.745,https://www.tinkoff.ru/invest/bonds/RU000A1053R3/
0,СПМК БО-02,RU000A106FN3,5,966.8,Низкий,30.885,1000.0,4,123.540,30.885,https://www.tinkoff.ru/invest/bonds/RU000A106FN3/
0,Агрофирма-племзавод Победа выпуск 1,RU000A102S23,4,939.1,Низкий,21.141,1000.0,4,84.564,21.141,https://www.tinkoff.ru/invest/bonds/RU000A102S23/
0,Alfa Bank,RU000A100GW5,11,8350000.0,Средний,254862.150,10000000.0,4,1019448.600,509724.300,https://www.tinkoff.ru/invest/bonds/RU000A100GW5/
...,...,...,...,...,...,...,...,...,...,...,...
0,Автодор БО-5 выпуск 3,RU000A106R53,34,1002.8,Средний,26.361,1000.0,4,105.444,210.888,https://www.tinkoff.ru/invest/bonds/RU000A106R53/
0,Газпромбанк 001P выпуск 14,RU000A101350,13,941.0,Высокий,30.189,1000.0,2,60.378,181.134,https://www.tinkoff.ru/invest/bonds/RU000A101350/
0,Вита Лайн 001Р выпуск 1,RU000A102200,10,982.9,Высокий,21.663,1000.0,4,86.652,43.326,https://www.tinkoff.ru/invest/bonds/RU000A102200/
0,Автодор выпуск 1,RU000A103NU9,35,897.0,Высокий,34.452,1000.0,2,68.904,585.684,https://www.tinkoff.ru/invest/bonds/RU000A103NU9/


In [165]:
df[df['Номинал'] <= 15000].sort_values(by=['Доходность(все время)'], ascending=False)

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Ссылка
0,Сегежа 002P выпуск 5,RU000A1053P7,166,899.4,Средний,46.110,1000.0,2,92.220,3827.130,https://www.tinkoff.ru/invest/bonds/RU000A1053P7/
0,РЖД 001Р выпуск 1,RU000A0JXN05,103,952.7,Высокий,39.237,1000.0,2,78.474,2001.087,https://www.tinkoff.ru/invest/bonds/RU000A0JXN05/
0,Альфа-Банк выпуск 18,RU000A0JX5W4,99,963.8,Высокий,40.281,1000.0,2,80.562,1973.769,https://www.tinkoff.ru/invest/bonds/RU000A0JX5W4/
0,Восточная Стивидорная Компания выпуск 4,RU000A106P06,58,968.1,Высокий,52.026,1000.0,2,104.052,1508.754,https://www.tinkoff.ru/invest/bonds/RU000A106P06/
0,Новые технологии,RU000A105DL4,49,1000.0,Высокий,58.551,1000.0,2,117.102,1405.224,https://www.tinkoff.ru/invest/bonds/RU000A105DL4/
...,...,...,...,...,...,...,...,...,...,...,...
0,ЯТЭК выпуск 1,RU000A102B48,1,995.2,Высокий,36.801,1000.0,2,73.602,0.000,https://www.tinkoff.ru/invest/bonds/RU000A102B48/
0,ПРОМОМЕД ДМ выпуск 2,RU000A103G91,1,973.8,27.11.2023,40.977,1000.0,2,81.954,0.000,https://www.tinkoff.ru/invest/bonds/RU000A103G91/
0,ФСК Россети БО-4,RU000A0ZYJ91,2,992.3,06.12.2023,16.443,1000.0,4,65.772,0.000,https://www.tinkoff.ru/invest/bonds/RU000A0ZYJ91/
0,Нафтатранс Плюс выпуск 3,RU000A102V51,5,952.0,Низкий,8.526,1000.0,12,102.312,0.000,https://www.tinkoff.ru/invest/bonds/RU000A102V51/


In [ ]:
df[df['Код'] == 'RU000A106WZ2']

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Ссылка
0,АПРИ Флай Плэнинг выпуск 3,RU000A106WZ2,47,102.0,Средний,45.501,1000.0,4,182.004,500.511,https://www.tinkoff.ru/invest/bonds/RU000A106WZ2/
